#### Hello im my notebook the name it pyspark_snowflake

**Set the configuration to allow spark session in primary node complete**

In [ ]:
!pip show pyspark 

In [ ]:
!java --version

In [ ]:
!whereis java

In [ ]:
!export JAVA_HOME=/opt/jdk

In [ ]:
!export PATH=$JAVA_HOME/bin:$PATH

In [ ]:
!java --version

In [ ]:
!which spark-submit

In [ ]:
from pyspark.sql import SparkSession

In [ ]:

spark = SparkSession.builder \
    .appName('sparksnowflake') \
    .config('spark.jars', '/home/hazem/snowflake_dbt_spark/snowflake_dbt/jars/mysql-connector-j-8.4.0.jar') \
    .enableHiveSupport() \
    .getOrCreate()


In [ ]:
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
sc

In [ ]:
spark.sparkContext.setLogLevel("ERROR")


In [ ]:
from pyspark.sql.functions import *

In [ ]:
from pyspark.sql.types import *

In [ ]:
import pyspark.sql.functions as fn 

In [ ]:
!pip install pandas 

In [ ]:
import pandas as pd

In [ ]:
!python3 -V

In [ ]:
!pip list

In [ ]:
data_list = [(1 , 'Ahmed','Omar' , 'Maadi' ,'male',['tennsi' , 'reading' ,'cooking']) , 
             (2 , 'Khaled','Montaser' , 'Helwan' ,'male',['football' , 'smoking' ,'cooking']),
             (3 , 'Hager','Ali' , 'Maadi' ,'female',['vollyball' , 'painting' ,'cooking']),
             (4 , 'Asmaa','Mansour' , 'Nacercity' ,None,['TV' , 'reading' ,'cooking'])              
            ]

In [ ]:
print(type(data_list))

In [ ]:
print(f"{data_list}" , end= ' ')

In [ ]:
i=0
for index , item in list(enumerate(data_list[1])):
    print(item)
   
    


In [ ]:
for item in data_list[0]:
    print(item )

In [ ]:
data_dict = {'ID':1 , 'Name':'Hazem' , 'Age':29} 

In [ ]:
words = 'Address'
if words in data_dict:
    data_dict[words] = 'Maadi'
else:
    data_dict[words] = 'None'
print(f'{data_dict}')

In [ ]:
data_dict.items()

In [ ]:
#('ID', 1)
for (index , word) in sorted(data_dict.items()):
    print(f'Index: {index:<8} | {word}')

In [ ]:
if data_dict[words]== 'Maadi':
    print(len(data_dict))

In [ ]:
from pyspark.sql.functions import *

In [ ]:
from pyspark.sql.types import *

In [ ]:
import pyspark.sql.functions as fn 

In [ ]:
# create programmatic schemaa 
schemaa = StructType([
    StructField('ID' , IntegerType() , True) ,
    StructField('First_Name' , StringType() , True) ,
    StructField('Last_Name' , StringType() , True) ,
    StructField('Address' , StringType() , True) ,
    StructField('Gender' , StringType() , True) ,
    StructField('Hobbits' , ArrayType(StringType()) , True) ,
])

In [ ]:
data_df = spark.createDataFrame(data_list , schema= schemaa)

In [ ]:
data_df.printSchema()


In [ ]:
data_df.dtypes

In [ ]:
data_df.show(n=4 , truncate=False)

In [ ]:
data_df_date_add = data_df.withColumn('hire_date' , current_timestamp())

In [ ]:
data_df_date_add.show(truncate=False)

In [ ]:
data_df_date_add.dtypes

In [ ]:
data_df_date_add_date = data_df_date_add.withColumn('hire_date' , col('hire_date').cast(DateType()))

In [ ]:
data_df_date_add_to_date  = data_df_date_add.withColumn('hire_date' , to_date(col('hire_date'))).show()

In [ ]:
data_df_date_add_to_timstamp = data_df_date_add.withColumn('hire_date' , to_timestamp(col('hire_date') , 'yyyy-dd-MM')).show(truncate=False)

In [ ]:
data_df_date_add_date.dtypes

In [ ]:
data_df_date_add_date.cache()

In [ ]:
data_df_date_add_date.show(truncate=False)

In [ ]:
data_df_date_add_date_select = data_df_date_add_date.select('ID' ,data_df_date_add_date['First_Name'].alias('first_name') , year(col('hire_date')).alias('year_date'))

In [ ]:
data_df_date_add_date_select.show()

In [ ]:
data_df_date_add_date_null = data_df_date_add_date.na.fill('N/A' ,subset=['Gender'])

In [ ]:
data_df_date_add_date_null.show()

In [ ]:
data_df_date_add_date.show()

In [ ]:
data_df_date_add_date_null_co = data_df_date_add_date.withColumn('Gender',coalesce(col('Gender'),lit('N/A')))

In [ ]:
data_df_date_add_date_null_co.show()

In [ ]:
data_df_date_add_date_null_co_hire = data_df_date_add_date_null_co.withColumn('hire_date', date_format(col('hire_date'),'dd-MM-yyyy'))

In [ ]:
data_df_date_add_date_null_co_hire.show()

In [ ]:
# Sliver Layer Check about the columns 
data_df_date_add_date_null_co_hire.select('*')\
.count()


In [ ]:
# codeing for bronz layer make a dta ingest
data_csv = spark.read.format('csv')\
.option('inferSchema' , 'True')\
.option('header' ,'True')\
.option('samplingRatio',0.001)\
.option('path', '/home/hazem/snowflake_dbt_spark/datasets/source_crm/cust_info.csv')\
.load()

In [ ]:
data_csv.dtypes

In [ ]:
data_csv.printSchema()

In [ ]:
data_csv.show(10)

In [ ]:
data_csv_count = data_csv.select('*')\
.count()

In [ ]:
# validation for bronz layer (Tracebility and dubging)
data_csv_count

In [ ]:
data_csv.dtypes

In [ ]:
# Sliver layer check about the data quality 
data_csv.show(5)

In [ ]:
# first column cst_id check the data about it (data qulity)
# when the checking end have issus what,s mean have duplication and redadency
data_csv_cst_id = data_csv.select('*')\
.groupBy('cst_id')\
.agg(fn.count('*').alias('count_number'))\
.where((col('cst_id').isNotNull()) & (col('count_number') > 1)).show()


In [ ]:
# Now solve this issus to get good quallity
# create tmp view this view virtual but needed to write sql operation 
data_csv.createOrReplaceTempView('customer_info')

In [ ]:
spark.sql('CREATE DATABASE IF NOT EXISTS Employee')

In [ ]:
spark.sql('USE Employee')

In [ ]:
data_csv_cst_id_edit = spark.sql('''
    SELECT * , ROW_NUMBER() over(PARTITION BY cst_id ORDER BY cst_id) as rn  FROM customer_info as crm
''').show()

In [ ]:
data_csv_cst_id_edit = spark.sql('''
    SELECT t.cst_id,t.cst_key,t.cst_firstname,t.cst_lastname,t.cst_marital_status,t.cst_gndr,t.cst_create_date
                                  FROM(
                SELECT * , ROW_NUMBER() over(PARTITION BY cst_id ORDER BY cst_id) as rn  FROM customer_info as crm
                )t
        WHERE t.rn = 1 AND t.cst_id IS NOT NULL 
''')

In [ ]:
data_csv_cst_id_edit.show()

### Now column in customer information remove the replica and redandancy 

In [ ]:
# Now check after making data qulity in column cst_id
data_csv_cst_id = data_csv_cst_id_edit.select('*')\
.groupBy('cst_id')\
.agg(fn.count('*').alias('count_number'))\
.where((col('cst_id').isNotNull()) & (col('count_number') > 1)).show()

In [ ]:
data_csv_cst_id_edit.show(2)

In [ ]:
data_csv_cst_cst_key_edit = data_csv_cst_id_edit.withColumn('cst_key' , regexp_extract(col('cst_key') , '(0.*)',1))

In [ ]:
data_csv_cst_cst_key_edit.show(2)

In [ ]:
# Remove Null's values 
data_csv_cst_cst_key_edit_null = data_csv_cst_cst_key_edit.select('*')\
.where(col('cst_key').isNotNull())

In [ ]:
data_csv_cst_cst_key_edit_null.show(2)

In [ ]:
# Now check about the data cleasing unwanted spaces
data_csv_cst_firstname_check = data_csv_cst_cst_key_edit_null.select('cst_firstname')\
.where(col('cst_firstname') != trim('cst_firstname')).count()

In [ ]:
# Now have a problem have 17 unwantwed spaces names 
data_csv_cst_firstname_check

In [ ]:
# Now solve this problem to handle unwanted spaces 
data_csv_cst_names_trim = data_csv_cst_cst_key_edit_null.withColumn('cst_firstname' , trim(col('cst_firstname')))\
    .withColumn('cst_lastname',trim(col('cst_lastname')))

In [ ]:
data_csv_cst_names_trim.show()

In [ ]:
data_csv_cst_firstname_check_after_trim = data_csv_cst_names_trim.select('cst_firstname')\
.where(col('cst_firstname') != trim('cst_firstname')).count()

In [ ]:
# Now data good quality by removing unwanted spaces from cst_firstname and cst_lastname
data_csv_cst_firstname_check_after_trim

In [ ]:
data_csv_cst_names_trim.show(5)

In [ ]:
# Now check about the data standralization and consistency 
data_csv_cst_marital_status = data_csv_cst_names_trim.select('cst_marital_status').distinct().show()

In [ ]:
data_csv_cst_names_trim.createOrReplaceTempView('customer_info_marital')

In [ ]:
spark.sql('SELECT * FROM customer_info_marital').show(2)

In [ ]:
data_csv_cst_gndr = data_csv_cst_names_trim.select('cst_gndr').distinct().show()

In [ ]:
data_csv_cst_marital_status_edit = spark.sql('''
    SELECT crm.cst_id , crm.cst_key,crm.cst_firstname,crm.cst_lastname,
                        CASE 
                            WHEN UPPER(TRIM(crm.cst_marital_status)) = 'M' THEN 'MARRIED'
                            WHEN UPPER(TRIM(crm.cst_marital_status)) = 'S' THEN 'SINGLE'                         
                            ELSE 'N/A'                         
                        END AS cst_marital_status , 
                        CASE
                            WHEN UPPER(TRIM(crm.cst_gndr)) = 'F' THEN 'FEMALE'
                            WHEN UPPER(TRIM(crm.cst_gndr)) = 'M' THEN 'MALE'
                            ELSE 'N/A'
                        END AS cst_gndr, 
                        crm.cst_create_date
                    FROM customer_info_marital as crm
''')

In [ ]:
data_csv_cst_marital_status_edit.show()

In [ ]:
data_csv_cst_marital_status_edit.dtypes

In [ ]:
data_csv_cst__date_format = data_csv_cst_marital_status_edit.withColumn('cst_create_date' , date_format(col('cst_create_date'),'dd-MM-yyyy'))

In [ ]:
data_csv_cst__date_format.dtypes

In [ ]:
data_csv_cst__date_format.withColumn('cst_create_date' , to_date(col('cst_create_date'),'dd-MM-yyyy')).show()

In [ ]:
data_csv_cst__date_format.show()

In [ ]:
data_csv_cst__date_format.dtypes

In [ ]:
spark.sql('USE Employee')

In [ ]:
data_csv_cst__date_format.write.mode('overwrite').saveAsTable('customer_infoo')

In [ ]:
spark.sql('USE Employee')
spark.sql('SELECT * FROM Employee.customer_infoo').show()

In [ ]:
data_csv_cst__date_format.createOrReplaceTempView('finale_customer_edit')

In [ ]:
data_csv_cst__date_format.show(2)

In [ ]:
data_customer_info = spark.sql('''
                SELECT    
                               ROW_NUMBER() over(order by fn.cst_create_date DESC) as customer_idd,
                               fn.cst_id as customer_id, 
                               fn.cst_key as customer_key,
                               fn.cst_firstname as customer_first_name,
                               fn.cst_lastname as customert_last_name,
                               fn.cst_marital_status as customer_marital_status,
                               fn.cst_gndr as customer_gender,
                               fn.cst_create_date as customer_date 
                               
                               FROM finale_customer_edit as fn 

''')

In [ ]:
data_customer_info.select('customer_gender').distinct().show()

In [ ]:
spark.sql('DROP TABLE IF EXISTS customer_infoo')

In [ ]:
# Save as managed table in spark-warehouse 
data_customer_info.write.mode('overwrite').saveAsTable('customer_infoo')

In [ ]:
spark.sql('SELECT * FROM customer_infoo').show()

In [ ]:
data_customer_info.write.format('parquet')\
.option('compression' , 'gzip')\
.option('path', '/home/hazem/snowflake_dbt_spark/scripts/files_output/')\
.mode('append')\
.save()

In [ ]:
data_customer_info.show(2)

In [ ]:
data_customer_info.write.format('jdbc') \
  .option('url','jdbc:mysql://mysql-db:3306/erp_spark') \
  .option("driver", "com.mysql.cj.jdbc.Driver")\
  .option('dbtable','customer_info') \
  .option('user','root') \
  .option('password','P@ssw0rd') \
  .mode('overwrite')\
  .save()
